<a href="https://colab.research.google.com/github/younesabdolmalaky/Pistachio/blob/main/dataprep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mahotas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 66.8 MB/s eta 0:00:00
  Using cached numpy-1.25.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.6 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.25.0 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.25.0 which is incompatible.


In [ ]:
!pip install imblearn
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d muratkokludataset/pistachio-image-dataset
! unzip /content/pistachio-image-dataset.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 64.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.25.0 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.25.0 which is incompatible.


ERROR: Could not find a version that satisfies the requirement mahotasa (from versions: none)
ERROR: No matching distribution found for mahotasa
mkdir: cannot create directory ‘/root/.kaggle’: File exists
 53% 14.0M/26.7M [00:00<00:00, 146MB/s]
100% 26.7M/26.7M [00:00<00:00, 190MB/s]
Archive:  /content/pistachio-image-dataset.zip
  inflating: Pistachio_Image_Dataset/Pistachio_16_Features_Dataset/Pistachio_16_Features_Dataset.arff  
  inflating: Pistachio_Image_Dataset/Pistachio_16_Features_Dataset/Pistachio_16_Features_Dataset.xls  
  inflating: Pistachio_Image_Dataset/Pistachio_16_Features_Dataset/Pistachio_16_Features_Dataset.xlsx  
  inflating: Pistachio_Image_Dataset/Pistachio_16_Features_Dataset/Pistachio_16_Features_Dataset_Citation_Request.txt  
  inflating: Pistachio_Image_Dataset/Pistachio_28_Features_Dataset/Pistachio_28_Features_Dataset.arff  
  inflating: Pistachio_Image_Dataset/Pistachio_28_Features_Dataset/Pistachio_28_Features_Dataset.xls  
  inflating: Pistachio_Image_D

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import mahotas
import h5py
import imageio as iio
import os
from os import listdir
from itertools import chain
import pickle
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from skimage.feature import hog
import random
from shutil import copyfile

In [2]:
df = pd.read_excel('/content/Pistachio_Image_Dataset/Pistachio_28_Features_Dataset/Pistachio_28_Features_Dataset.xls')
y = df['Class']
df = df.drop('Class' , axis = 1)

In [3]:
def read_images(dirname , df):
  hue_features_all = []
  haralick_features_all = []
  features28 = []
  names = []
  for image in os.listdir(dirname):
      number_str = image.split(" ")[-1].split(".")[0]
      number_str = number_str.replace("(", "").replace(")", "")  # remove parentheses
      number = int(number_str)
      img = iio.imread(dirname+'/' +image)
      image_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      hue_features = cv2.HuMoments(cv2.moments(image_gray)).flatten()
      haralick_features = mahotas.features.haralick(image_gray).mean(axis=0)
      hue_features_all.append(hue_features)
      haralick_features_all.append(haralick_features)
      features28.append(df.iloc[number].values)
  return np.concatenate((hue_features_all ,haralick_features_all , features28) , axis =1)

In [4]:
Kirmizi_Pistachio_train = read_images('/content/drive/MyDrive/Pistachio/train/Kirmizi_Pistachio' , df)
Siirt_Pistachio_train = read_images('/content/drive/MyDrive/Pistachio/train/Siirt_Pistachio' , df)
Kirmizi_Pistachio_test = read_images('/content/drive/MyDrive/Pistachio/test/Kirmizi_Pistachio' , df)
Siirt_Pistachio_test = read_images('/content/drive/MyDrive/Pistachio/test/Siirt_Pistachio' , df)

<ipython-input-3-9383c7d0f1cc>:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = iio.imread(dirname+'/' +image)


In [8]:
Kirmizi_Pistachio_train = pd.DataFrame(Kirmizi_Pistachio_train)
Kirmizi_Pistachio_train['names'] = os.listdir('/content/drive/MyDrive/Pistachio/train/Kirmizi_Pistachio')

Siirt_Pistachio_train = pd.DataFrame(Siirt_Pistachio_train)
Siirt_Pistachio_train['names'] = os.listdir('/content/drive/MyDrive/Pistachio/train/Siirt_Pistachio')

Kirmizi_Pistachio_test = pd.DataFrame(Kirmizi_Pistachio_test)
Kirmizi_Pistachio_test['names'] = os.listdir('/content/drive/MyDrive/Pistachio/test/Kirmizi_Pistachio')

Siirt_Pistachio_test = pd.DataFrame(Siirt_Pistachio_test)
Siirt_Pistachio_test['names'] = os.listdir('/content/drive/MyDrive/Pistachio/test/Siirt_Pistachio')

In [9]:
with open('/content/drive/MyDrive/Pistachio/Kirmizi_Pistachio_train.pickle', 'wb') as f:
    pickle.dump(Kirmizi_Pistachio_train, f)

with open('/content/drive/MyDrive/Pistachio/Siirt_Pistachio_train.pickle', 'wb') as f:
    pickle.dump(Siirt_Pistachio_train, f)

with open('/content/drive/MyDrive/Pistachio/Kirmizi_Pistachio_test.pickle', 'wb') as f:
    pickle.dump(Kirmizi_Pistachio_test, f)

with open('/content/drive/MyDrive/Pistachio/Siirt_Pistachio_test.pickle', 'wb') as f:
    pickle.dump(Siirt_Pistachio_test, f)